In [1]:
from __future__ import print_function

import os
import sys
import glob
import h5py
import numpy as np
import math


import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorboard
import tensorboardX
from torch.utils.tensorboard import SummaryWriter


from log import Logger
from data import  trainDataset, testDataset, trainlabelDataset,testlabelDataset
from util import r2, mse, rmse, mae, pp_mse, pp_rmse, pp_mae
from model import autoencoder_999, autoencoder_333

def to_img(x):   # image size 
    x = x.view(x.size(0), 1, 64, 64)
    return x




if not os.path.exists('./gal_img1001'):
    os.mkdir('./gal_img1001')

    
dataset= trainlabelDataset()
dataloader= DataLoader(dataset=dataset, batch_size=64,shuffle=True,drop_last=True)

test_dataset = testlabelDataset()
test_dataloader= DataLoader(dataset=test_dataset, batch_size=64,shuffle=True,drop_last=True)


writer = SummaryWriter("run1001/exp411",)  ################################################### change name 

num_epochs =20000
batch_size = 64
learning_rate = 5e-1

model = autoencoder_333().cuda()   ############################################################## AE model 
model.load_state_dict(torch.load('gal_img1001/exp401_7700.pth'))    ###

criterion_mean = nn.L1Loss(reduction='mean')
criterion_none = nn.L1Loss(reduction='none')



#scheduler 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1000,4000], gamma=0.1)

In [15]:
for epoch in range(num_epochs):
    total_loss = 0.0
    total_mse = 0.0 
    total_recon=0.0
    total_latent=0.0
    
    num_examples = 0.0
    test_num_examples=0.0
    
    
    test_total_loss = 0.0    
    test_total_mse=0.0
    test_total_recon=0.0
    test_total_latent=0.0
    model.train()
    for data in dataloader:
        img,label= [x.type(torch.float32).cuda() for x in data]
        img = img.view(img.size(0), 1,64,64)

       # print(img.shape)
       # print("",img.sum())
       # print("",img[0].sum())
        # forward
        output, z = model(img)
        z=z.view(z.size(0),14*14)
       # print("output ",output.shape)
       # print("z ",z.shape)
    ################################################## Loss function with regularizing Z ########################
        
        loss= (( criterion_none(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1))).mean()) + (criterion_none(z[:,:7], label)).mean() /(5e3) 

        loss_recon=(( criterion_none(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1))).mean())
        loss_latent=(criterion_none(z[:,:7], label)).mean() /(5e3)
        
        
        MSE_loss = nn.MSELoss()(output, img)
        batch_size = img.size(0)
        total_loss += loss.item() * batch_size
        total_mse += MSE_loss.item() * batch_size
        total_recon+= loss_recon.item() * batch_size
        total_latent+= loss_latent.item() * batch_size

        num_examples += batch_size

        
        optimizer.zero_grad()
    # backward
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    model.eval()
    for data in test_dataloader:
        test_img,test_label= [x.type(torch.float32).cuda() for x in data]

        test_img = test_img.view(test_img.size(0), 1,64,64)
       # print(img.shape)
        

        # forward
        test_output,test_z = model(test_img)
        test_z=test_z.view(test_z.size(0),14*14)
                
       #print("output ",output.shape)
       # test_loss = criterion(test_output, test_img) + criterion(z[:,:7], label) /(1e)  #  + 1e-5*  criterion(test_z[:,:7], test_label) 
        test_loss= (( criterion_none(test_output, test_img)/(test_img.sum(dim=3).sum(dim=2).sum(dim=1))).mean())  + (criterion_none(test_z[:,:7], test_label) /(5e3)).mean() 
        
        test_loss_recon= (( criterion_none(test_output, test_img)/(test_img.sum(dim=3).sum(dim=2).sum(dim=1))).mean()) 
        test_loss_latent= (criterion_none(test_z[:,:7], test_label) /(5e3)).mean() 
        
        test_MSE_loss = nn.MSELoss()(test_output, test_img)
        batch_size = test_img.size(0)
        test_total_loss += test_loss.item() * batch_size
        test_total_mse += test_MSE_loss.item() * batch_size
        test_total_recon+= test_loss_recon.item() * batch_size
        test_total_latent+= test_loss_latent.item() * batch_size


        test_num_examples += batch_size

    writer.add_scalar('Loss/train',total_loss / num_examples,epoch)
    writer.add_scalar('Mse/train', total_mse / num_examples,epoch)   
    writer.add_scalar('Recon/train', total_recon / num_examples,epoch)        
    writer.add_scalar('Latent/train', total_latent / num_examples,epoch)        
    writer.add_scalar('Loss/test',test_total_loss / test_num_examples,epoch)
    writer.add_scalar('Mse/test', test_total_mse / test_num_examples,epoch)
    writer.add_scalar('Recon/test', test_total_recon / test_num_examples,epoch)        
    writer.add_scalar('Latent/test', test_total_latent / test_num_examples,epoch)   
    
    
    print('epoch [{}/{}], loss:{:.6f}, MSE_loss:{:.6f}, recon_loss:{:.6f}, latent_loss:{:.6f}'
          .format(epoch + 1, num_epochs, total_loss / num_examples, total_mse/ num_examples, total_recon / num_examples, total_latent/ num_examples))     
    
    print('epoch [{}/{}], test_loss:{:.6f}, test_MSE_loss:{:.6f}, test_recon_loss:{:.6f}, test_latent_loss:{:.6f}'
          .format(epoch + 1, num_epochs, test_total_loss / num_examples, test_total_mse/ num_examples, test_total_recon / num_examples, test_total_latent/ num_examples))   
    
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        test_x = to_img(test_img.cpu().data)    ########## change name 
        test_x_hat = to_img(test_output.cpu().data)
        torch.save(x, './gal_img1001/exp411_x_{}.pt'.format(epoch))
        torch.save(x_hat, './gal_img1001/exp411_x_hat_{}.pt'.format(epoch))
        torch.save(test_x, './gal_img1001/exp411_test_x_{}.pt'.format(epoch))
        torch.save(test_x_hat, './gal_img1001/exp411_test_x_hat_{}.pt'.format(epoch))
        torch.save(model.state_dict(), './gal_img1001/exp411_{}.pth'.format(epoch))     

epoch [1/20000], loss:0.362304, MSE_loss:0.000046, recon_loss:0.026757, latent_loss:0.335547
epoch [1/20000], test_loss:0.118701, test_MSE_loss:0.001831, test_recon_loss:0.019489, test_latent_loss:0.099212
epoch [2/20000], loss:0.381489, MSE_loss:0.000158, recon_loss:0.045949, latent_loss:0.335539
epoch [2/20000], test_loss:0.102719, test_MSE_loss:0.000002, test_recon_loss:0.003482, test_latent_loss:0.099238
epoch [3/20000], loss:0.370519, MSE_loss:0.000065, recon_loss:0.034992, latent_loss:0.335528
epoch [3/20000], test_loss:0.102927, test_MSE_loss:0.000003, test_recon_loss:0.003659, test_latent_loss:0.099268
epoch [4/20000], loss:0.375468, MSE_loss:0.001725, recon_loss:0.039939, latent_loss:0.335529
epoch [4/20000], test_loss:0.104087, test_MSE_loss:0.000003, test_recon_loss:0.004854, test_latent_loss:0.099233
epoch [5/20000], loss:0.344406, MSE_loss:0.000010, recon_loss:0.008875, latent_loss:0.335531
epoch [5/20000], test_loss:0.100111, test_MSE_loss:0.000000, test_recon_loss:0.0009

KeyboardInterrupt: 

In [11]:
label.shape

torch.Size([64, 7])

In [12]:
label[0]

tensor([17.8529,  2.8329,  1.9098,  0.2186,  0.9737,  0.6552,  2.3758],
       device='cuda:0')

In [13]:
test_z[0,:6]

tensor([0., 0., 0., 0., 0., 0.], device='cuda:0', grad_fn=<SliceBackward>)

In [4]:
loss= (( criterion_none(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1))).mean())* batch_size  + criterion_mean(z[:,:7], label) /(1e10) 
loss

tensor(0.6114, device='cuda:0', grad_fn=<AddBackward0>)

In [8]:
(( criterion_none(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1))).mean())

tensor(0.0095, device='cuda:0', grad_fn=<MeanBackward0>)

In [6]:
 criterion_mean(z[:,:7], label) /(1e10)

tensor(0.0007, device='cuda:0', grad_fn=<DivBackward0>)

In [7]:
criterion_mean(output, img)

tensor(0.0019, device='cuda:0', grad_fn=<L1LossBackward>)

In [11]:
(( criterion_mean(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1)))).mean()

tensor(0.0096, device='cuda:0', grad_fn=<MeanBackward0>)

In [9]:
label[0][6]

tensor(2.3758, device='cuda:0')

In [10]:
 z.shape

torch.Size([64, 196])

In [ ]:
criterion(z[:,:7], label)/1e7

In [ ]:
img.shape

In [ ]:
img[0].shape

In [ ]:
img[0].sum()

In [ ]:
img[0].max()

In [ ]:
img.sum()

In [ ]:
img.sum()

In [ ]:
criterion(output, img)

In [ ]:
4.7/1878

In [ ]:
criterion = nn.L1Loss()
a=criterion(output, img); 
a

In [14]:
z.shape

torch.Size([64, 196])

In [ ]:
b=output.padding(2)

In [ ]:
criterion = nn.L1Loss(reduction='none')
a=criterion(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1)); 
a.shape
a.mean()   # should i use a.mean * batch size as loss here? 

In [ ]:
a.shape

In [11]:
test_label.shape

torch.Size([64, 7])

In [8]:
torch.log(test_label[0][0])


tensor(16.9483, device='cuda:0')

In [13]:
test_label[0][0]

tensor(80532144., device='cuda:0')

In [ ]:
o=criterion(output, img);
o.shape


In [ ]:
s=(img.sum(dim=3).sum(dim=2).sum(dim=1));
s.shape

In [ ]:
(o/s).shape

In [ ]:
(o/s)[0].mean()

In [ ]:
(o[0]/s[0]).mean()

In [ ]:
(o.view(64,64*64)).sum(dim=1)


In [ ]:
s.size()

In [ ]:
img[0].sum()

In [ ]:
o.size()

In [ ]:
(o[0]/s[0]).min()

In [ ]:
(o[0]/s[0]).mean()

In [ ]:
criterion = nn.L1Loss(reduction='none')
a=criterion(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1)); 
a[0].mean()

In [ ]:
o

In [ ]:
a.sum(dim=3).sum(dim=2).sum(dim=1)

In [ ]:
criterion = nn.L1Loss(reduction='mean')

b=torch.zeros(1)
for i in range(64): 
    a=criterion(output[i]/img[i].sum(), img[i]/img[i].sum())
    b=b+a 
    
b    # b is not equal to a 

In [ ]:
criterion = nn.L1Loss(reduction='mean')
criterion(output, img)


In [ ]:
b/64   # b/64 =a 

In [ ]:
criterion(output[0]/img[0].sum(), img[0]/img[0].sum())

In [ ]:
summ=(img.sum(dim=3).sum(dim=2).sum(dim=1));
summ.mean()

In [ ]:
img.shape

In [ ]:
img.sum().shape

In [ ]:
img.sum(dim=3).sum(dim=2).sum(dim=1)  # img 64*1*64*64 

In [ ]:
img[1].sum()

In [ ]:
criterion = nn.L1Loss()
criterion(output, img)

In [ ]:
output.shape

In [ ]:
img[0].sum()

In [ ]:
img.sum()


In [ ]:
 criterion(output, img)/2064

In [ ]:
b/64 * 1e3

In [ ]:
b/64

In [ ]:
b/64

In [ ]:
b=torch.zeros(1)
for i in range(64): 
    a=criterion(output[i], img[i])
    b=b+a 

In [ ]:
b/img.sum()

In [ ]:
img[0].sum()

In [ ]:
img.sum()

In [ ]:
criterion(output/img.sum(), img/img.sum())

In [ ]:
criterion(z[:,:7]

In [ ]:
 criterion(output, img)

In [ ]:
print('epoch [{}/{}], loss:{:.6f}, MSE_loss:{:.6f}, recon_loss:{:.6f}, latent_loss:{:.6f}'
          .format(epoch + 1, num_epochs, total_loss / num_examples, total_mse/ num_examples, total_recon / num_examples, total_latent/ num_examples))   

In [ ]:
        loss= ( criterion_none(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1))) #  .mean())  #* batch_size # + criterion_mean(z[:,:7], label) /(1e9) 


In [ ]:
criterion_none(output, img).shape